In [8]:
import xarray as xr
import pandas as pd
import dask.dataframe as dd
import re
import requests
import pickle as pk
import os
import gc
#!pip install netcdf4 == 1.5.0

In [2]:
# make the output directory
new_dir = '/home/jovyan/data/botpt/RS03CCAL-MJ03E-06-BOTPTA302/'
if not os.path.isdir(new_dir):
    try:
        os.makedirs(new_dir)
    except OSError:
        if os.path.exists(new_dir):
            pass
        else:
            raise

In [9]:
url = 'https://opendap.oceanobservatories.org/thredds/catalog/ooi/jazlynnatalie12@gmail.com/20191111T212931803Z-RS03ECAL-MJ03E-06-BOTPTA302-streamed-botpt_nano_sample/catalog.html'
# Sensor E url = 'https://opendap.oceanobservatories.org/thredds/catalog/ooi/dax.soule@qc.cuny.edu/20191022T034014801Z-RS03ECAL-MJ03E-06-BOTPTA302-streamed-botpt_nano_sample/catalog.html'
# url = 'https://opendap.oceanobservatories.org/thredds/catalog/ooi/jazlynnatalie12@gmail.com/20191106T032013294Z-RS03ECAL-MJ03E-06-BOTPTA302-streamed-botpt_nano_sample/catalog.html'
# url = 'https://opendap.oceanobservatories.org/thredds/catalog/ooi/jazlynnatalie12@gmail.com/20191106T031235918Z-RS03ECAL-MJ03E-06-BOTPTA302-streamed-botpt_nano_sample/catalog.html'
# url = 'https://opendap.oceanobservatories.org/thredds/catalog/ooi/jazlynnatalie12@gmail.com/20191106T031235918Z-RS03ECAL-MJ03E-06-BOTPTA302-streamed-botpt_nano_sample/catalog.html'
# url = '/home/jovyan/data/botpt/RS03CCAL-MJ03E-06-BOTPTA302/deployment0001_RS03ECAL-MJ03E-06-BOTPTA302-streamed-botpt_nano_sample_20160918T120000-20160925T235959.950000'
# url = 'https://opendap.oceanobservatories.org/thredds/catalog/ooi/dax.soule@qc.cuny.edu/20191022T034014801Z-RS03ECAL-MJ03E-06-BOTPTA302-streamed-botpt_nano_sample/catalog.html'
# url = 'https://opendap.oceanobservatories.org/thredds/catalog/ooi/jazlynnatalie12@gmail.com/20191106T032013294Z-RS03ECAL-MJ03E-06-BOTPTA302-streamed-botpt_nano_sample/catalog.html'
# sensor B url = 'https://opendap.oceanobservatories.org/thredds/catalog/ooi/dax.soule@qc.cuny.edu/20191022T174216826Z-RS03ASHS-MJ03B-09-BOTPTA304-streamed-botpt_nano_sample/catalog.html'
# url = 'https://opendap.oceanobservatories.org/thredds/catalog/ooi/jazlynnatalie12@gmail.com/20191106T032013294Z-RS03ECAL-MJ03E-06-BOTPTA302-streamed-botpt_nano_sample/catalog.html'
# url = 'https://opendap.oceanobservatories.org/thredds/catalog/ooi/dax.soule@qc.cuny.edu/20191026T023346674Z-RS03CCAL-MJ03F-05-BOTPTA301-streamed-botpt_nano_sample/catalog.html'
# Sensor D:url = 'https://opendap.oceanobservatories.org/thredds/catalog/ooi/dax.soule@qc.cuny.edu/20191015T040958136Z-RS03INT2-MJ03D-06-BOTPTA303-streamed-botpt_nano_sample/catalog.html'

In [10]:
tds_url = 'https://opendap.oceanobservatories.org/thredds/dodsC'
datasets = requests.get(url).text
urls = re.findall(r'href=[\'"]?([^\'" >]+)', datasets)
x = re.findall(r'(ooi/.*?.nc)', datasets)
for i in x:
    if i.endswith('.nc') == False:
        x.remove(i)
for i in x:
    try:
        float(i[-4])
    except:
        x.remove(i)
datasets = [os.path.join(tds_url, i + '#fillmismatch') for i in x]

In [14]:
# datasets

# pd routines

In [15]:
%%time
for i in datasets:
    ds = xr.open_dataset(i)
    ds = ds.swap_dims({'obs': 'time'})

    pressure_min = pd.DataFrame()
    pressure_min['bottom_pressure'] = ds['bottom_pressure'].to_pandas().resample('T').mean()
    del pressure_min.index.name

    pressure_min = pressure_min.dropna()

    out = new_dir + i.split('/')[-1][:-16] + '_resampled' + '.nc'
    ds = xr.Dataset.from_dataframe(pressure_min)
    ds.to_netcdf(out)

ValueError: cannot swap from dimension 'obs' because it is not an existing dimension

In [17]:
i=datasets[1]
ds = xr.open_dataset(i)
ds = ds.swap_dims({'obs': 'time'})

pressure_min = pd.DataFrame()
pressure_min['bottom_pressure'] = ds['bottom_pressure'].to_pandas().resample('T').mean()
del pressure_min.index.name

pressure_min = pressure_min.dropna()

out = new_dir + i.split('/')[-1][:-16]
out

'/home/jovyan/data/botpt/RS03CCAL-MJ03E-06-BOTPTA302/deployment0001_RS03ECAL-MJ03E-06-BOTPTA302-streamed-botpt_nano_sample_20170208T180000-20170217T115959.950000'

## Full NetCDF download

In [18]:
%%time
import subprocess
download_data = 'wget -r -l1 -nd -nc -np -e robots=off -A.nc --no-check-certificate https://opendap.oceanobservatories.org/thredds/catalog/ooi/dax.soule@qc.cuny.edu/20191022T034014801Z-RS03ECAL-MJ03E-06-BOTPTA302-streamed-botpt_nano_sample/'
subprocess.call(download_data, shell=True) 

CPU times: user 1.5 ms, sys: 14.8 ms, total: 16.3 ms
Wall time: 169 ms


8

## xr routine 

In [19]:
%%time
for i in datasets:
    ds = xr.open_mfdataset([i], parallel =True, combine='by_coords')
    ds = ds.swap_dims({'obs': 'time'})
    ds= ds['bottom_pressure'].to_dataset()
    ds=ds.resample(time='T').mean()
    out = 'minute_mean_data/' + i.split('/')[-1][:-3] + '_resampled_test' + '.nc'
    ds.to_netcdf(out)

PermissionError: [Errno 13] Permission denied: b'/home/jovyan/botpt/notebooks/minute_mean_data/deployment0001_RS03ECAL-MJ03E-06-BOTPTA302-streamed-botpt_nano_sample_20170217T120000-20170224T235959.950000.nc#fillmisma_resampled_test.nc'

In [7]:
from dask.distributed import Client

client = Client("tcp://10.0.128.212:46185")
client

OSError: Timed out trying to connect to 'tcp://10.0.128.212:46185' after 10 s: Timed out trying to connect to 'tcp://10.0.128.212:46185' after 10 s: connect() didn't finish in time

In [ ]:
%%time
for i in datasets:
    ds = xr.open_mfdataset([i], parallel =True, combine='by_coords'
    out = 'new_dir' + i.split('/')[-1][:-3] + 'dask'
    ds.to_netcdf(out)